In [8]:
using JuMP, DataFrames, Gurobi, FileIO, TextParse, CSVFiles, Clp

# Set-working directory
#DIR = "C:\\Users\\will-\\GoogleDrive\\UCBerkeley\\Research\\Papers\\2018 Off-grid\\"
#DIR = "C:\\Users\\Will\\GoogleDrive\\UCBerkeley\\Research\\Papers\\2018 Off-grid\\Analysis\\"
DIR = "G:\\Team Drives\\grid_defect_data\\"
OUT = "Analysis\\out"
INPUT = "Analysis\\in"

##### CREATE MODEL RUN ######
# Set constants
LOAD_SHED = [0.25, 0]
BAT_COST = 500 # $/kWh
PV_COST = 3000 # $/kW
BAT_EFF = 0.92
#annual rates
int_rate = 0.03 # percentage interest rate
bat_life = 20 # years
sol_life = 25 # years
BAT_RATE = int_rate / (1 - (1+int_rate)^(-bat_life))
PV_RATE = int_rate / (1 - (1+int_rate)^(-sol_life));


In [9]:
# identify geography IDs to loop through
ID_G = load(DIR * INPUT * "\\optimization_list.csv") |> DataFrame

,,row_in_2,county,state,lat.x,lon,timezone,elevation,tz,ID,lat.y,long
1,1,1,san bernardino,ca,34.6109,-116.128,-8,572,US/Pacific,690150,34.3,-116.167
2,2,2,fisher,tx,32.7624,-100.324,-6,577,US/Central,690190,32.433,-99.85
3,3,2,mitchell,tx,32.3606,-100.901,-6,653,US/Central,690190,32.433,-99.85
4,4,2,runnels,tx,31.8286,-100.06,-6,540,US/Central,690190,32.433,-99.85
5,5,2,scurry,tx,32.7344,-100.945,-6,712,US/Central,690190,32.433,-99.85
6,6,2,stonewall,tx,33.1593,-100.239,-6,540,US/Central,690190,32.433,-99.85
7,7,2,nolan,tx,32.2643,-100.382,-6,761,US/Central,690190,32.433,-99.85
8,8,2,taylor,tx,32.3494,-99.8453,-6,586,US/Central,690190,32.433,-99.85
9,9,3,island,wa,48.1326,-122.564,-8,77,US/Pacific,690230,48.35,-122.667
10,10,3,san juan,wa,48.5698,-122.956,-8,0,US/Pacific,690230,48.35,-122.667


In [3]:
script = "C:\\Users\\wgorman\\Desktop\\grid_defection\\"
include(script * "optimization_model.jl")

solar_opt (generic function with 1 method)

In [4]:
#Run Model
timing = @elapsed for i in 1:(nrow(ID_G)*2*3)
    
    result = solar_opt(ID_G, LOAD_SHED, BAT_COST, BAT_RATE, BAT_EFF, PV_COST, PV_RATE, DIR, INPUT, i) 
    
    if (i == 1)
        global results = result
    else
        append!(results, result)
    end
    
    #if (i == 2)
    #    global outcomes = outcome
    #else
    #    append!(outcomes, outcome)
    #end
    
   # if (i == 2)
   #     global stati = status
   # else
   #     append!(stati, status)
   # end

    print(i)
end

Academic license - for non-commercial use only
Optimize a model with 236522 rows, 394203 columns and 904534 nonzeros
Coefficient statistics:
  Matrix range     [5e-06, 1e+00]
  Objective range  [3e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 3e+04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 2 rows and 78842 columns
Presolve time: 1.54s
Presolved: 236520 rows, 315361 columns, 825689 nonzeros

Ordering time: 0.07s

Barrier statistics:
 Dense cols : 2
 AA' NZ     : 5.103e+05
 Factor NZ  : 3.798e+06 (roughly 250 MBytes of memory)
 Factor Ops : 7.370e+07 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   8.32001644e+04  0.00000000e+00  3.16e+07 0.00e+00  1.28e+02     2s
   1   1.92458570e+05  4.06877161e+03  8.37e+06 1.48e-01  5.31e+01     2s
   2   2.32915686e+05  6.25283317e+0

2Academic license - for non-commercial use only
Optimize a model with 236522 rows, 394203 columns and 904534 nonzeros
Coefficient statistics:
  Matrix range     [5e-06, 1e+00]
  Objective range  [3e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 2e+04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 2 rows and 78842 columns
Presolve time: 1.29s
Presolved: 236520 rows, 315361 columns, 825689 nonzeros

Ordering time: 0.04s

Barrier statistics:
 Dense cols : 2
 AA' NZ     : 5.103e+05
 Factor NZ  : 3.798e+06 (roughly 250 MBytes of memory)
 Factor Ops : 7.370e+07 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.21830400e+04  0.00000000e+00  1.60e+07 0.00e+00  6.63e+01     2s
   1   9.74317269e+04  2.48221913e+03  4.25e+06 1.51e-01  2.75e+01     2s
   2   1.18386748e+05  3.85926683e+

4Academic license - for non-commercial use only
Optimize a model with 236522 rows, 394203 columns and 904534 nonzeros
Coefficient statistics:
  Matrix range     [5e-06, 1e+00]
  Objective range  [3e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-01, 4e+04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 2 rows and 78842 columns
Presolve time: 1.52s
Presolved: 236520 rows, 315361 columns, 825689 nonzeros

Ordering time: 0.04s

Barrier statistics:
 Dense cols : 2
 AA' NZ     : 5.103e+05
 Factor NZ  : 3.798e+06 (roughly 250 MBytes of memory)
 Factor Ops : 7.370e+07 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.29269760e+05  0.00000000e+00  4.91e+07 0.00e+00  1.96e+02     2s
   1   2.99282359e+05  5.37138064e+03  1.30e+07 1.46e-01  8.14e+01     2s
   2   3.61273648e+05  8.05725351e+

6Academic license - for non-commercial use only
Optimize a model with 236522 rows, 394203 columns and 904507 nonzeros
Coefficient statistics:
  Matrix range     [2e-05, 1e+00]
  Objective range  [3e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 3e+04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 2 rows and 78842 columns
Presolve time: 1.29s
Presolved: 236520 rows, 315361 columns, 825662 nonzeros

Ordering time: 0.04s

Barrier statistics:
 Dense cols : 2
 AA' NZ     : 5.103e+05
 Factor NZ  : 3.798e+06 (roughly 250 MBytes of memory)
 Factor Ops : 7.370e+07 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   8.18815399e+04  0.00000000e+00  3.11e+07 0.00e+00  1.27e+02     2s
   1   1.88345434e+05  4.12516352e+03  8.41e+06 1.49e-01  5.28e+01     2s
   2   2.29526291e+05  6.47963238e+

8Academic license - for non-commercial use only
Optimize a model with 236522 rows, 394203 columns and 904507 nonzeros
Coefficient statistics:
  Matrix range     [2e-05, 1e+00]
  Objective range  [3e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 2e+04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 2 rows and 78842 columns
Presolve time: 1.25s
Presolved: 236520 rows, 315361 columns, 825662 nonzeros

Ordering time: 0.04s

Barrier statistics:
 Dense cols : 2
 AA' NZ     : 5.103e+05
 Factor NZ  : 3.798e+06 (roughly 250 MBytes of memory)
 Factor Ops : 7.370e+07 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.17104009e+04  0.00000000e+00  1.59e+07 0.00e+00  6.59e+01     2s
   1   9.58729366e+04  2.51010994e+03  4.27e+06 1.52e-01  2.74e+01     2s
   2   1.17174155e+05  3.98351304e+

10Academic license - for non-commercial use only
Optimize a model with 236522 rows, 394203 columns and 904507 nonzeros
Coefficient statistics:
  Matrix range     [2e-05, 1e+00]
  Objective range  [3e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-01, 4e+04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 2 rows and 78842 columns
Presolve time: 1.36s
Presolved: 236520 rows, 315361 columns, 825662 nonzeros

Ordering time: 0.04s

Barrier statistics:
 Dense cols : 2
 AA' NZ     : 5.103e+05
 Factor NZ  : 3.798e+06 (roughly 250 MBytes of memory)
 Factor Ops : 7.370e+07 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.28327971e+05  0.00000000e+00  4.88e+07 0.00e+00  1.95e+02     2s
   1   2.95399104e+05  5.41639145e+03  1.32e+07 1.47e-01  8.13e+01     2s
   2   3.59008150e+05  8.33594070e

12Academic license - for non-commercial use only
Optimize a model with 236522 rows, 394203 columns and 904571 nonzeros
Coefficient statistics:
  Matrix range     [3e-06, 1e+00]
  Objective range  [3e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 3e+04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 2 rows and 78842 columns
Presolve time: 1.06s
Presolved: 236520 rows, 315361 columns, 825726 nonzeros

Ordering time: 0.04s

Barrier statistics:
 Dense cols : 2
 AA' NZ     : 5.104e+05
 Factor NZ  : 3.798e+06 (roughly 250 MBytes of memory)
 Factor Ops : 7.369e+07 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   8.37002085e+04  0.00000000e+00  3.18e+07 0.00e+00  1.31e+02     2s
   1   2.02719250e+05  4.05950864e+03  8.54e+06 1.47e-01  5.32e+01     2s
   2   2.38958307e+05  6.46123656e

14Academic license - for non-commercial use only
Optimize a model with 236522 rows, 394203 columns and 904571 nonzeros
Coefficient statistics:
  Matrix range     [3e-06, 1e+00]
  Objective range  [3e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 2e+04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 2 rows and 78842 columns
Presolve time: 1.11s
Presolved: 236520 rows, 315361 columns, 825726 nonzeros

Ordering time: 0.05s

Barrier statistics:
 Dense cols : 2
 AA' NZ     : 5.104e+05
 Factor NZ  : 3.798e+06 (roughly 250 MBytes of memory)
 Factor Ops : 7.369e+07 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.23520716e+04  0.00000000e+00  1.61e+07 0.00e+00  6.75e+01     2s
   1   1.02634696e+05  2.49644756e+03  4.31e+06 1.50e-01  2.75e+01     2s
   2   1.21273924e+05  4.02175570e

16Academic license - for non-commercial use only
Optimize a model with 236522 rows, 394203 columns and 904571 nonzeros
Coefficient statistics:
  Matrix range     [3e-06, 1e+00]
  Objective range  [3e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-01, 4e+04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 2 rows and 78842 columns
Presolve time: 1.23s
Presolved: 236520 rows, 315361 columns, 825726 nonzeros

Ordering time: 0.04s

Barrier statistics:
 Dense cols : 2
 AA' NZ     : 5.104e+05
 Factor NZ  : 3.798e+06 (roughly 250 MBytes of memory)
 Factor Ops : 7.369e+07 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.30176283e+05  0.00000000e+00  4.95e+07 0.00e+00  2.01e+02     2s
   1   3.15188370e+05  5.35156634e+03  1.33e+07 1.45e-01  8.19e+01     2s
   2   3.75445829e+05  8.16173899e

971.862317384

In [19]:
save(DIR * OUT * "\\outcomes_no_out_constraint_pv_vos.csv", outcomes)

In [55]:
save(DIR * OUT * "\\results.csv", results)

In [105]:
results

,pv,storage,shed_frac,id,load,ann_cost,solar_tot,pv_curtail,shed_tot
1,8.66796,21.2503,0.25,BASE_allen_ky,1.15748e5,2960.55,1.06575e5,11753.7,28936.9
2,20.565,434.978,0.0001,BASE_allen_ky,1.15748e5,23787.9,2.52852e5,1.09421e5,11.5748


In [109]:
results

,pv,storage,shed_frac,id,load,ann_cost,solar_tot,pv_curtail,shed_tot
1,8.69143,21.1262,0.25,BASE_allen_ky,1.15748e5,2207.4,1.06863e5,10217.2,28936.9
2,29.8739,385.304,0.0001,BASE_allen_ky,1.15748e5,18096.0,3.67307e5,2.32581e5,11.5748


In [114]:
results

,pv,storage,shed_frac,id,load,ann_cost,solar_tot,pv_curtail,shed_tot
1,8.69143,21.1262,0.25,BASE_allen_ky,1.15748e5,2207.4,1.06863e5,12056.2,28936.9
2,29.8739,385.304,0.0001,BASE_allen_ky,1.15748e5,18096.0,3.67307e5,2.05812e5,11.5748


In [120]:
results

,pv,storage,shed_frac,id,load,ann_cost,solar_tot,pv_curtail,shed_tot
1,8.69143,21.1262,0.25,BASE_allen_ky,1.15748e5,2207.4,1.06863e5,7276.39,28936.9
2,29.8739,385.304,0.0001,BASE_allen_ky,1.15748e5,18096.0,3.67307e5,1.98823e5,11.5748


In [10]:
results

,pv,storage,shed_frac,id,load,ann_cost,solar_tot,pv_curtail,shed_tot
1,29.8739,385.304,0.0001,BASE_allen_ky,1.15748e5,18096.0,3.67307e5,31468.8,11.5748


In [130]:
stati

:Optimal

In [33]:
results

,pv,storage,id
1,29.8739,385.304,BASE_allen_ky


In [5]:
results

,pv,storage,id
1,8.69441,21.1067,BASE_allen_ky
2,29.8739,385.304,BASE_allen_ky
3,4.67354,11.8533,LOW_allen_ky
4,16.6369,210.627,LOW_allen_ky
5,12.961,29.5727,HIGH_allen_ky
6,39.9225,518.236,HIGH_allen_ky
7,8.75682,21.0823,BASE_barren_ky
8,46.7431,222.614,BASE_barren_ky
9,4.70054,11.8634,LOW_barren_ky
10,25.6514,121.845,LOW_barren_ky
